In [2]:

import os, sys, queue, time, re
import sounddevice as sd
from google.cloud import speech
from google.oauth2 import service_account
from six.moves import queue as six_queue
import unidecode, spacy

# ---------- spaCy (futuro análisis) ----------
nlp = spacy.load("es_core_news_sm")

# ---------- Parsing "junta N a ±M grados" ----------
def parse_command(text: str):
    t = unidecode.unidecode(text).lower()
    t = re.sub(r"\ba\s+mas\s+(\d+)", r"+\1", t)
    t = re.sub(r"\ba\s+menos\s+(\d+)", r"-\1", t)
    m = re.search(r"junta\s+(\d+).*?([+-]?\d+)\s*grados", t)
    if m:
        return int(m.group(1)), int(m.group(2))
    return None, None

# ---------- Google STT setup ----------
KEYFILE = "STT_demo.json"
creds   = service_account.Credentials.from_service_account_file(KEYFILE)
client  = speech.SpeechClient(credentials=creds)
RATE, CHUNK = 16000, 1600  # 100 ms

recognition_config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=RATE,
    language_code="es-ES",
    enable_automatic_punctuation=True,
)
streaming_config = speech.StreamingRecognitionConfig(
    config=recognition_config, interim_results=True
)

# ---------- Cola de audio ----------
audio_q: six_queue.Queue[bytes] = queue.Queue()

def sd_callback(indata, frames, time_info, status):
    if status: print("[SD]", status)
    audio_q.put(indata.tobytes())

def audio_gen():
    while True:
        chunk = audio_q.get()
        if chunk is None:
            return
        yield speech.StreamingRecognizeRequest(audio_content=chunk)

# ---------- PySide6 GUI ----------
from PySide6.QtCore    import Qt, QThread, Signal
from PySide6.QtWidgets import (
    QApplication, QWidget, QLabel, QVBoxLayout
)

class STTWorker(QThread):
    liveText   = Signal(str)
    finalText  = Signal(str)
    commandSig = Signal(str)  # texto "Junta 4 → -20°"

    def run(self):
        with sd.RawInputStream(samplerate=RATE, blocksize=CHUNK,
            dtype="int16", channels=1, callback=sd_callback):
            requests  = audio_gen()
            responses = client.streaming_recognize(streaming_config, requests)
            for resp in responses:
                if not resp.results:
                    continue
                res = resp.results[0]
                txt = res.alternatives[0].transcript.strip()
                if res.is_final:
                    self.finalText.emit(txt)
                    j, a = parse_command(txt)
                    if j is not None:
                        self.commandSig.emit(f"Junta {j} → {a}°")
                else:
                    self.liveText.emit(txt)

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Reconocimiento de Voz – Demo GUI")
        self.setFixedSize(430, 220)

        self.stateLbl  = QLabel("🔴 Inactivo")
        self.liveLbl   = QLabel("LIVE: …")
        self.finalLbl  = QLabel("FIN : …")
        self.cmdLbl    = QLabel("Comando: —")

        for lbl in (self.stateLbl, self.liveLbl, self.finalLbl, self.cmdLbl):
            lbl.setStyleSheet("font-size:14px;")
            lbl.setWordWrap(True)

        vbox = QVBoxLayout(self)
        vbox.addWidget(self.stateLbl)
        vbox.addWidget(self.liveLbl)
        vbox.addWidget(self.finalLbl)
        vbox.addWidget(self.cmdLbl)

        # Hilo de reconocimiento
        self.worker = STTWorker()
        self.worker.liveText.connect(self.on_live)
        self.worker.finalText.connect(self.on_final)
        self.worker.commandSig.connect(self.on_command)
        self.worker.start()

    # ---------- slots ----------
    def on_live(self, txt):
        self.liveLbl.setText(f"LIVE: {txt[:60]}")
        self.stateLbl.setText("🟡 Escuchando")

    def on_final(self, txt):
        self.finalLbl.setText(f"FIN : {txt[:60]}")
        self.stateLbl.setText("🟢 Procesando")

    def on_command(self, cmdtxt):
        self.cmdLbl.setText(f"Comando: {cmdtxt}")
        self.stateLbl.setText("✅ Comando detectado")

# ---------- main ----------
if __name__ == "__main__":
    app = QApplication(sys.argv)
    mw  = MainWindow()
    mw.show()
    sys.exit(app.exec())

RuntimeError: Please destroy the QApplication singleton before creating a new QApplication instance.